In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import sys
from pymongo import MongoClient
import json
from sqlalchemy import create_engine

### Extract CSVs into DataFrames

In [2]:
Airplane_crashes_file = "./Resources/Airplane_Crashes.csv"
Airplane_crashes_df = pd.read_csv(Airplane_crashes_file, encoding = "ISO-8859-1")
Airplane_crashes_df.head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary
0,09/17/1908,17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly..."
1,07/12/1912,06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...
2,08/06/1913,NaN,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...
3,09/09/1913,18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...
4,10/17/1913,10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...


In [3]:
Airplane_crashes_df['Time'] = Airplane_crashes_df['Time'].replace(np.nan, '00:00') 
Airplane_crashes_df['Time'] = Airplane_crashes_df['Time'].str.replace('c: ', '')
Airplane_crashes_df['Time'] = Airplane_crashes_df['Time'].str.replace('c:', '')
Airplane_crashes_df['Time'] = Airplane_crashes_df['Time'].str.replace('c', '')
Airplane_crashes_df['Time'] = Airplane_crashes_df['Time'].str.replace('12\'20', '12:20')
Airplane_crashes_df['Time'] = Airplane_crashes_df['Time'].str.replace('18.40', '18:40')
Airplane_crashes_df['Time'] = Airplane_crashes_df['Time'].str.replace('0943', '09:43')
Airplane_crashes_df['Time'] = Airplane_crashes_df['Time'].str.replace('22\'08', '22:08')
Airplane_crashes_df['Time'] = Airplane_crashes_df['Time'].str.replace('114:20', '00:00')

In [4]:
Airplane_crashes_df['Time'] = Airplane_crashes_df['Date'] + ' ' + Airplane_crashes_df['Time']

In [5]:
Airplane_crashes_df['idate']=pd.to_datetime(Airplane_crashes_df['Time'])
Airplane_crashes_df['idate'].head()

0   1908-09-17 17:18:00
1   1912-07-12 06:30:00
2   1913-08-06 00:00:00
3   1913-09-09 18:30:00
4   1913-10-17 10:30:00
Name: idate, dtype: datetime64[ns]

In [6]:
Airplane_crashes_df.head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,idate
0,09/17/1908,09/17/1908 17:18,"Fort Myer, Virginia",Military - U.S. Army,NaN,Demonstration,Wright Flyer III,NaN,1,2.0,1.0,0.0,"During a demonstration flight, a U.S. Army fly...",1908-09-17 17:18:00
1,07/12/1912,07/12/1912 06:30,"AtlantiCity, New Jersey",Military - U.S. Navy,NaN,Test flight,Dirigible,NaN,NaN,5.0,5.0,0.0,First U.S. dirigible Akron exploded just offsh...,1912-07-12 06:30:00
2,08/06/1913,08/06/1913 00:00,"Victoria, British Columbia, Canada",Private,-,NaN,Curtiss seaplane,NaN,NaN,1.0,1.0,0.0,The first fatal airplane accident in Canada oc...,1913-08-06 00:00:00
3,09/09/1913,09/09/1913 18:30,Over the North Sea,Military - German Navy,NaN,NaN,Zeppelin L-1 (airship),NaN,NaN,20.0,14.0,0.0,The airship flew into a thunderstorm and encou...,1913-09-09 18:30:00
4,10/17/1913,10/17/1913 10:30,"Near Johannisthal, Germany",Military - German Navy,NaN,NaN,Zeppelin L-2 (airship),NaN,NaN,30.0,30.0,0.0,Hydrogen gas which was being vented was sucked...,1913-10-17 10:30:00


In [7]:
print('Date ranges from ' + str(Airplane_crashes_df.idate.min()) + ' to ' + str(Airplane_crashes_df.idate.max()))

Date ranges from 1908-09-17 17:18:00 to 2009-06-08 00:00:00


In [8]:
Airplane_crashes_drop=Airplane_crashes_df.dropna()
Airplane_crashes_drop.head()

,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,idate
208,01/19/1930,01/19/1930 18:23,"Oceanside, California",Maddux Airlines,7,"Aqua Caliente, Mexico - Los Angeles",Ford 5-AT-C Tri Motor,NC9689,5-AT-046,16.0,16.0,0.0,"While en route to Los Angeles, the pilot, flyi...",1930-01-19 18:23:00
236,03/31/1931,03/31/1931 10:45,"Bazaar, Kansas",Trans Continental and Western Air,599,Kansas City - Wichita - Los Angeles,Fokker F10A Trimotor,NC-999,1063,8.0,8.0,0.0,"Shortly after taking off from Kansas City, one...",1931-03-31 10:45:00
334,08/31/1934,08/31/1934 23:42,"Amazonia, Missouri",Rapid Air Transport,6,Omaha - St. Joseph,Stinson SM-6000B,NC10809,5004,5.0,5.0,0.0,The plane crashed about 11 miles from St. Jose...,1934-08-31 23:42:00
354,05/06/1935,05/06/1935 03:30,"Atlanta, Missouri",Trans Continental and Western Air,6,Los Angeles - Albuquerque - Kanasas City - Wa...,Douglas DC-2-112,NC13785,1295,14.0,5.0,0.0,The plane crashed while en route from Albuquer...,1935-05-06 03:30:00
365,08/14/1935,08/14/1935 23:45,"Near Gilmer, Texas",Delta Air Lines,4,Dallas - Atlanta,Stinson Model A,NC14599,9103,4.0,4.0,0.0,Crashed 3 miles south of Gilmer. The outboard ...,1935-08-14 23:45:00


In [9]:
Airplane_crashes_drop.insert(0, 'ID', range(1000, 1000+len(Airplane_crashes_drop)))
Airplane_crashes_drop.head()

,ID,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,idate
208,1000,01/19/1930,01/19/1930 18:23,"Oceanside, California",Maddux Airlines,7,"Aqua Caliente, Mexico - Los Angeles",Ford 5-AT-C Tri Motor,NC9689,5-AT-046,16.0,16.0,0.0,"While en route to Los Angeles, the pilot, flyi...",1930-01-19 18:23:00
236,1001,03/31/1931,03/31/1931 10:45,"Bazaar, Kansas",Trans Continental and Western Air,599,Kansas City - Wichita - Los Angeles,Fokker F10A Trimotor,NC-999,1063,8.0,8.0,0.0,"Shortly after taking off from Kansas City, one...",1931-03-31 10:45:00
334,1002,08/31/1934,08/31/1934 23:42,"Amazonia, Missouri",Rapid Air Transport,6,Omaha - St. Joseph,Stinson SM-6000B,NC10809,5004,5.0,5.0,0.0,The plane crashed about 11 miles from St. Jose...,1934-08-31 23:42:00
354,1003,05/06/1935,05/06/1935 03:30,"Atlanta, Missouri",Trans Continental and Western Air,6,Los Angeles - Albuquerque - Kanasas City - Wa...,Douglas DC-2-112,NC13785,1295,14.0,5.0,0.0,The plane crashed while en route from Albuquer...,1935-05-06 03:30:00
365,1004,08/14/1935,08/14/1935 23:45,"Near Gilmer, Texas",Delta Air Lines,4,Dallas - Atlanta,Stinson Model A,NC14599,9103,4.0,4.0,0.0,Crashed 3 miles south of Gilmer. The outboard ...,1935-08-14 23:45:00


In [10]:
crashes=Airplane_crashes_drop
crashes.head()

,ID,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,idate
208,1000,01/19/1930,01/19/1930 18:23,"Oceanside, California",Maddux Airlines,7,"Aqua Caliente, Mexico - Los Angeles",Ford 5-AT-C Tri Motor,NC9689,5-AT-046,16.0,16.0,0.0,"While en route to Los Angeles, the pilot, flyi...",1930-01-19 18:23:00
236,1001,03/31/1931,03/31/1931 10:45,"Bazaar, Kansas",Trans Continental and Western Air,599,Kansas City - Wichita - Los Angeles,Fokker F10A Trimotor,NC-999,1063,8.0,8.0,0.0,"Shortly after taking off from Kansas City, one...",1931-03-31 10:45:00
334,1002,08/31/1934,08/31/1934 23:42,"Amazonia, Missouri",Rapid Air Transport,6,Omaha - St. Joseph,Stinson SM-6000B,NC10809,5004,5.0,5.0,0.0,The plane crashed about 11 miles from St. Jose...,1934-08-31 23:42:00
354,1003,05/06/1935,05/06/1935 03:30,"Atlanta, Missouri",Trans Continental and Western Air,6,Los Angeles - Albuquerque - Kanasas City - Wa...,Douglas DC-2-112,NC13785,1295,14.0,5.0,0.0,The plane crashed while en route from Albuquer...,1935-05-06 03:30:00
365,1004,08/14/1935,08/14/1935 23:45,"Near Gilmer, Texas",Delta Air Lines,4,Dallas - Atlanta,Stinson Model A,NC14599,9103,4.0,4.0,0.0,Crashed 3 miles south of Gilmer. The outboard ...,1935-08-14 23:45:00


In [11]:
crashes.dtypes

ID                       int32
Date                    object
Time                    object
Location                object
Operator                object
Flight #                object
Route                   object
Type                    object
Registration            object
cn/In                   object
Aboard                 float64
Fatalities             float64
Ground                 float64
Summary                 object
idate           datetime64[ns]
dtype: object

In [12]:
crashes['Date'] = pd.to_datetime(crashes['Date'])
crashes.head()

C:\Users\sstat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ID,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,idate
208,1000,1930-01-19,01/19/1930 18:23,"Oceanside, California",Maddux Airlines,7,"Aqua Caliente, Mexico - Los Angeles",Ford 5-AT-C Tri Motor,NC9689,5-AT-046,16.0,16.0,0.0,"While en route to Los Angeles, the pilot, flyi...",1930-01-19 18:23:00
236,1001,1931-03-31,03/31/1931 10:45,"Bazaar, Kansas",Trans Continental and Western Air,599,Kansas City - Wichita - Los Angeles,Fokker F10A Trimotor,NC-999,1063,8.0,8.0,0.0,"Shortly after taking off from Kansas City, one...",1931-03-31 10:45:00
334,1002,1934-08-31,08/31/1934 23:42,"Amazonia, Missouri",Rapid Air Transport,6,Omaha - St. Joseph,Stinson SM-6000B,NC10809,5004,5.0,5.0,0.0,The plane crashed about 11 miles from St. Jose...,1934-08-31 23:42:00
354,1003,1935-05-06,05/06/1935 03:30,"Atlanta, Missouri",Trans Continental and Western Air,6,Los Angeles - Albuquerque - Kanasas City - Wa...,Douglas DC-2-112,NC13785,1295,14.0,5.0,0.0,The plane crashed while en route from Albuquer...,1935-05-06 03:30:00
365,1004,1935-08-14,08/14/1935 23:45,"Near Gilmer, Texas",Delta Air Lines,4,Dallas - Atlanta,Stinson Model A,NC14599,9103,4.0,4.0,0.0,Crashed 3 miles south of Gilmer. The outboard ...,1935-08-14 23:45:00


In [13]:
crashes.dtypes

ID                       int32
Date            datetime64[ns]
Time                    object
Location                object
Operator                object
Flight #                object
Route                   object
Type                    object
Registration            object
cn/In                   object
Aboard                 float64
Fatalities             float64
Ground                 float64
Summary                 object
idate           datetime64[ns]
dtype: object

In [14]:
crashes['year'] = crashes['Date'].dt.year
crashes.head()

C:\Users\sstat\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ID,Date,Time,Location,Operator,Flight #,Route,Type,Registration,cn/In,Aboard,Fatalities,Ground,Summary,idate,year
208,1000,1930-01-19,01/19/1930 18:23,"Oceanside, California",Maddux Airlines,7,"Aqua Caliente, Mexico - Los Angeles",Ford 5-AT-C Tri Motor,NC9689,5-AT-046,16.0,16.0,0.0,"While en route to Los Angeles, the pilot, flyi...",1930-01-19 18:23:00,1930
236,1001,1931-03-31,03/31/1931 10:45,"Bazaar, Kansas",Trans Continental and Western Air,599,Kansas City - Wichita - Los Angeles,Fokker F10A Trimotor,NC-999,1063,8.0,8.0,0.0,"Shortly after taking off from Kansas City, one...",1931-03-31 10:45:00,1931
334,1002,1934-08-31,08/31/1934 23:42,"Amazonia, Missouri",Rapid Air Transport,6,Omaha - St. Joseph,Stinson SM-6000B,NC10809,5004,5.0,5.0,0.0,The plane crashed about 11 miles from St. Jose...,1934-08-31 23:42:00,1934
354,1003,1935-05-06,05/06/1935 03:30,"Atlanta, Missouri",Trans Continental and Western Air,6,Los Angeles - Albuquerque - Kanasas City - Wa...,Douglas DC-2-112,NC13785,1295,14.0,5.0,0.0,The plane crashed while en route from Albuquer...,1935-05-06 03:30:00,1935
365,1004,1935-08-14,08/14/1935 23:45,"Near Gilmer, Texas",Delta Air Lines,4,Dallas - Atlanta,Stinson Model A,NC14599,9103,4.0,4.0,0.0,Crashed 3 miles south of Gilmer. The outboard ...,1935-08-14 23:45:00,1935


In [22]:
crashes.shape

(1000, 16)

In [16]:
crashes.to_csv("crashes.csv")

In [17]:
crashes.dtypes

ID                       int32
Date            datetime64[ns]
Time                    object
Location                object
Operator                object
Flight #                object
Route                   object
Type                    object
Registration            object
cn/In                   object
Aboard                 float64
Fatalities             float64
Ground                 float64
Summary                 object
idate           datetime64[ns]
year                     int64
dtype: object

### Create database connection

In [23]:
#client=MongoClient() #replace database name with your name

In [24]:
#db=client.crashes

In [25]:
#crash_details=db.crash_details

In [26]:
#records_ = crashes.to_dict(orient='records')
#db.crash_details.insert_many(records_) #replace the word database with the name of your databaseaccident_count_per_year